In [121]:
import pandas as pd

In [122]:
df = pd.read_csv(r"C:\Users\capor\Downloads\dataset(in).csv")

In [123]:
df

,Unnamed: 0,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,318,34,Private,209297,HS-grad,9,Never-married,Machine-op-inspct,Not-in-family,Black,Male,0,2001,40,United-States,<=50K
1,11608,32,Private,107417,HS-grad,9,Never-married,Other-service,Not-in-family,White,Male,0,0,37,United-States,<=50K
2,2362,59,Local-gov,296253,Masters,14,Divorced,Prof-specialty,Unmarried,White,Female,8614,0,60,United-States,>50K
3,17387,44,Federal-gov,102238,HS-grad,9,Divorced,Craft-repair,Unmarried,White,Male,0,0,40,United-States,<=50K
4,22358,27,?,190650,Bachelors,13,Never-married,?,Unmarried,Asian-Pac-Islander,Male,0,0,25,Philippines,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29300,17852,21,Private,306114,Some-college,10,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
29301,20240,40,?,224361,9th,5,Divorced,?,Unmarried,White,Female,0,0,5,Cuba,<=50K
29302,1433,31,Private,251659,Some-college,10,Married-civ-spouse,Other-service,Husband,Asian-Pac-Islander,Male,0,1485,55,?,>50K
29303,25247,41,Private,39581,Some-college,10,Divorced,Adm-clerical,Not-in-family,Black,Female,0,0,24,El-Salvador,<=50K


In [124]:
df.drop(['Unnamed: 0','education.num', 'capital.gain', 'capital.loss'], axis=1, inplace=True)
import numpy as np

df.replace("?", np.nan, inplace=True)

In [125]:
df = df.dropna()

In [126]:
df

,age,workclass,fnlwgt,education,marital.status,occupation,relationship,race,sex,hours.per.week,native.country,income
0,34,Private,209297,HS-grad,Never-married,Machine-op-inspct,Not-in-family,Black,Male,40,United-States,<=50K
1,32,Private,107417,HS-grad,Never-married,Other-service,Not-in-family,White,Male,37,United-States,<=50K
2,59,Local-gov,296253,Masters,Divorced,Prof-specialty,Unmarried,White,Female,60,United-States,>50K
3,44,Federal-gov,102238,HS-grad,Divorced,Craft-repair,Unmarried,White,Male,40,United-States,<=50K
5,43,Local-gov,163434,Bachelors,Married-civ-spouse,Protective-serv,Husband,White,Male,55,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...
29298,63,Private,294009,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,45,United-States,>50K
29299,28,Private,319768,Masters,Married-civ-spouse,Sales,Husband,White,Male,45,France,>50K
29300,21,Private,306114,Some-college,Never-married,Adm-clerical,Own-child,White,Male,20,United-States,<=50K
29303,41,Private,39581,Some-college,Divorced,Adm-clerical,Not-in-family,Black,Female,24,El-Salvador,<=50K


In [127]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'marital.status',
       'occupation', 'relationship', 'race', 'sex', 'hours.per.week',
       'native.country', 'income'],
      dtype='object')

In [128]:
df["age"].value_counts()

34    762
35    761
36    756
33    750
31    750
     ... 
84      8
82      7
83      4
85      3
88      3
Name: age, Length: 71, dtype: int64

In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27160 entries, 0 to 29304
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             27160 non-null  int64 
 1   workclass       27160 non-null  object
 2   fnlwgt          27160 non-null  int64 
 3   education       27160 non-null  object
 4   marital.status  27160 non-null  object
 5   occupation      27160 non-null  object
 6   relationship    27160 non-null  object
 7   race            27160 non-null  object
 8   sex             27160 non-null  object
 9   hours.per.week  27160 non-null  int64 
 10  native.country  27160 non-null  object
 11  income          27160 non-null  object
dtypes: int64(3), object(9)
memory usage: 2.7+ MB


In [130]:
from sklearn.preprocessing import LabelEncoder

# Lista de columnas a codificar
columnas_categoricas = [
    "workclass", "marital.status", "occupation", "relationship",
    "race", "sex", "native.country", "income", "education",
]

for col in columnas_categoricas:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])  # Transforma la columna# Guarda el encoder por si se necesita revertir después

# Ver los primeros registros transformados
print(df.head())

   age  workclass  fnlwgt  education  marital.status  occupation  \
0   34          2  209297         11               4           6   
1   32          2  107417         11               4           7   
2   59          1  296253         12               0           9   
3   44          0  102238         11               0           2   
5   43          1  163434          9               2          10   

   relationship  race  sex  hours.per.week  native.country  income  
0             1     2    1              40              38       0  
1             1     4    1              37              38       0  
2             4     4    0              60              38       1  
3             4     4    1              40              38       0  
5             0     4    1              55              38       1  


C:\Users\capor\AppData\Local\Temp\ipykernel_23920\4193447720.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = le.fit_transform(df[col])  # Transforma la columna# Guarda el encoder por si se necesita revertir después
C:\Users\capor\AppData\Local\Temp\ipykernel_23920\4193447720.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = le.fit_transform(df[col])  # Transforma la columna# Guarda el encoder por si se necesita revertir después
C:\Users\capor\AppData\Local\Temp\ipykernel_23920\41

In [131]:
df

,age,workclass,fnlwgt,education,marital.status,occupation,relationship,race,sex,hours.per.week,native.country,income
0,34,2,209297,11,4,6,1,2,1,40,38,0
1,32,2,107417,11,4,7,1,4,1,37,38,0
2,59,1,296253,12,0,9,4,4,0,60,38,1
3,44,0,102238,11,0,2,4,4,1,40,38,0
5,43,1,163434,9,2,10,0,4,1,55,38,1
...,...,...,...,...,...,...,...,...,...,...,...,...
29298,63,2,294009,9,2,9,0,4,1,45,38,1
29299,28,2,319768,12,2,11,0,4,1,45,9,1
29300,21,2,306114,15,4,0,3,4,1,20,38,0
29303,41,2,39581,15,0,0,1,2,0,24,7,0


In [132]:
df["income"].value_counts()

0    20369
1     6791
Name: income, dtype: int64

In [133]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'marital.status',
       'occupation', 'relationship', 'race', 'sex', 'hours.per.week',
       'native.country', 'income'],
      dtype='object')

## Modelo

In [134]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import pickle  # Usaremos pickle para guardar y cargar los objetos

def train_and_evaluate_models(df, save_path="modelo_y_escalador.pkl"):
    # Definir X e y
    X = df.drop(columns=["income"])
    y = df["income"]

    # Dividir el conjunto de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Escalar los datos
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Definir modelos e hiperparámetros
    models = {
        'Random Forest': RandomForestClassifier(random_state=0)
    }

    param_grid = {
        'Random Forest': {
            'n_estimators': [65, 70, 75, 100],
            'max_depth': [6, 7],
            'min_samples_split': [2, 3],
            'min_samples_leaf': [1, 2, 3],
        }
    }

    # Resultados
    results = []
    
    # Realizar Grid Search para cada modelo
    for name, model in models.items():
        print(f"Training {name}...")
        grid_search = GridSearchCV(model, param_grid[name], cv=5, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train_scaled, y_train)

        # Obtener mejor modelo y parámetros
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_
        best_score = grid_search.best_score_

        # Evaluar en conjunto de prueba
        y_pred = best_model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)

        # Registrar resultados
        results.append({
            'Model': name,
            'Best Parameters': best_params,
            'Train Accuracy (CV)': best_score,
            'Test Accuracy': accuracy,
            'Classification Report': report
        })

        print(f"{name} - Best Parameters: {best_params}")
        print(f"{name} - Train Accuracy (CV): {best_score}")
        print(f"{name} - Test Accuracy: {accuracy}\n")
        print(report)

        # Guardar el mejor modelo y el escalador en un solo archivo .pkl
        with open(save_path, "wb") as f:
            pickle.dump({"modelo": best_model, "escalador": scaler}, f)

        print(f"Modelo y escalador guardados en '{save_path}'")

    # Convertir resultados a DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# Llamar a la función y guardar el modelo y escalador en un solo archivo
results_df = train_and_evaluate_models(df, save_path="modelo_y_escalador.pkl")

Training Random Forest...
Random Forest - Best Parameters: {'max_depth': 7, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 100}
Random Forest - Train Accuracy (CV): 0.8240516179050668
Random Forest - Test Accuracy: 0.8252945508100147

              precision    recall  f1-score   support

           0       0.87      0.91      0.89      4120
           1       0.67      0.55      0.60      1312

    accuracy                           0.83      5432
   macro avg       0.77      0.73      0.75      5432
weighted avg       0.82      0.83      0.82      5432

Modelo y escalador guardados en 'modelo_y_escalador.pkl'


In [135]:
import pickle

# Cargar el modelo y el escalador desde el archivo .pkl
with open("modelo_y_escalador.pkl", "rb") as f:
    data = pickle.load(f)

modelo_cargado = data["modelo"]
escalador_cargado = data["escalador"]

print("Modelo y escalador cargados correctamente.")

Modelo y escalador cargados correctamente.


In [136]:
modelo_cargado

RandomForestClassifier(max_depth=7, min_samples_leaf=3, random_state=0)

In [137]:
escalador_cargado

StandardScaler()